# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (sparse implementation)

In [ ]:
import logging
import numpy as np
import h5py
import jax
import jax.numpy as jnp
from jax.experimental.sparse import bcsr_dot_general
from fastdla.sparse_pauli_sum import SparsePauliSumArray, SparsePauliSum
from fastdla.lie_closure import lie_closure, orthogonalize
from fastdla.generators.z2lgt_hva import (z2lgt_hva_generators, z2lgt_gauss_projector,
                                          z2lgt_u1_projector, z2lgt_translation_projector,
                                          z2lgt_symmetry_eigenspace)

logging.basicConfig(level=logging.WARNING)
logging.getLogger('fastdla').setLevel(logging.INFO)
jax.config.update('jax_enable_x64', True)
SparsePauliSum.switch_impl('fast')

In [ ]:
data_file = ''
max_workers = 48

### Choice of symmetry sector

In [ ]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
u1_total_charge = 0
t_jphase = 0

### Full list of HVA generators

In [ ]:
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

### Using SparsePauliSums

In [ ]:
# Compute the DLA of the full space
# WARNING: This can take *forever* for num_fermions >= 4
dla_full = lie_closure(generators_full, min_tasks=200, max_workers=max_workers)
print(f'DLA dimension is {len(dla_full)}')

In [ ]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, u1_total_charge)
t_projector = z2lgt_translation_projector(num_fermions, t_jphase)

assert (gauss_projector.commutator(u1_projector).num_terms == 0
        and gauss_projector.commutator(t_projector).num_terms == 0
        and u1_projector.commutator(t_projector).num_terms == 0)

In [ ]:
# Symmetry projectors commute
# (t_projector commutes with the rest only for specific t_jphase values (such as 0))
# -> Subspace can be narrowed with simple matrix multiplication of the projectors
symm_projector = gauss_projector @ u1_projector @ t_projector

# Furthermore, the full projector commutes with all HVA generators
# -> Generators can be projected onto the symmetry subspace by one-side application of the
#    projectors. The resulting operators are the HVA generators in the subspace.
ops = []
for op in generators_full:
    assert op.commutator(symm_projector).num_terms == 0
    ops.append((op @ symm_projector).normalize())
generators_symm = SparsePauliSumArray(ops)

In [ ]:
# Count the number of linearly independent generators when limited to the subspace
# (same sanitization is performed in lie_closure())
basis = SparsePauliSumArray([generators_symm[0]])
for gen in generators_symm[1:]:
    orth = orthogonalize(gen, basis)
    if orth.num_terms != 0:
        basis.append(orth.normalize())
print(f'{len(basis)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = lie_closure(generators_symm, min_tasks=200, max_workers=max_workers)
print(f'Subspace DLA dimension is {len(dla)}')

### Using matrices

In [ ]:
generators_csr = generators_full.to_matrices(sparse=True, npmod=jnp)
symm_eigenspace = z2lgt_symmetry_eigenspace(gauss_eigvals, u1_total_charge, t_jphase, npmod=jnp)
ops = []
for op in generators_csr:
    op = bcsr_dot_general(op, symm_eigenspace, dimension_numbers=(([1], [0]), ([], [])))
    ops.append(symm_eigenspace.conjugate().T @ op)
generators_symm = jnp.array(ops)

In [ ]:
# Compute the DLA of the subspace
dla_arr = lie_closure(generators_symm)
print(f'Subspace DLA dimension is {len(dla_arr)}')

### Save the calculation result

In [ ]:
if data_file:
    with h5py.File(data_file, 'a') as out:
        if f'nf={num_fermions}' not in out:
            nf_group = out.create_group(f'nf={num_fermions}')
            for gname, oplist in [
                ('generators_full', generators_full),
                ('dla_full', dla_full),
                ('generators_symm', generators_symm),
                ('dla_symm', dla)
            ]:
                group = nf_group.create_group(gname)
                group.create_dataset('indices', data=oplist.indices)
                group.create_dataset('coeffs', data=oplist.coeffs)
                group.create_dataset('indptr', data=oplist.ptrs)

            nf_group.create_dataset('dla_symm/gauss_eigvals', data=gauss_eigvals)
            nf_group.create_dataset('dla_symm/u1_total_charge', data=u1_total_charge)
            nf_group.create_dataset('dla_symm/t_jphase', data=t_jphase)